In [1]:
from ngsolve import *
from netgen import gui

In [2]:
from netgen.geom2d import SplineGeometry
geo = SplineGeometry()
geo.AddRectangle( (0, 0), (1, 1),
                 bcs = ("bottom", "right", "top", "left"))
mesh = Mesh( geo.GenerateMesh(maxh=0.125))
Draw(mesh)
order = 4

In [3]:
from math import pi
b = CoefficientFunction((1+sin(4*pi*y),2))
Draw(b,mesh,"wind")
from ngsolve.internal import visoptions; visoptions.scalfunction = "wind:0"
ubnd = IfPos(x-0.125,IfPos(0.625-x,1+cos(8*pi*x),0),0)

In [4]:
VT = L2(mesh,order=order)
u,v = VT.TnT()
c = BilinearForm(VT, nonassemble=True)

In [5]:
n = specialcf.normal(mesh.dim)
upw_flux = b*n * IfPos(b*n, u, u.Other(bnd=ubnd))
dS = dx(element_boundary=True)
c += - b * grad(v) * u * dx + upw_flux * v * dS



In [6]:
gfu = GridFunction(VT);
Draw(gfu,mesh,"u",sd=4,min=-0.1,max=2.1,autoscale=False)
res = gfu.vec.CreateVector()

# Explicit Euler

In [7]:
#Operator application (equivalent to assemble and mult but faster)
res.data = c.mat * gfu.vec

In [8]:
invm = VT.Mass(1).Inverse()
print(invm)



Print base-matrix



In [9]:
%%time
#better: %%timeit
t = 0; gfu.vec[:] = 0
dt = 2e-4
tend = 0.6

while t < tend-0.5*dt:
    res.data = invm @ c.mat * gfu.vec
    gfu.vec.data -= dt * res
    t += dt
    Redraw(blocking=True)

CPU times: user 4.57 s, sys: 185 ms, total: 4.75 s
Wall time: 4.77 s


# Implizite Euler

In [18]:
M = BilinearForm(VT)
M += u*v*dx
M_star = BilinearForm(VT)

M_star = M + c

TypeError: unsupported operand type(s) for +: 'ngsolve.comp.BilinearForm' and 'ngsolve.comp.BilinearForm'